# Reunion Credit Risk Prediction


## Part II - ML Models

#### Business Objective:

It is far more important for an entity while doing credit risk prediction to make sure applicants who are at high credit risk are not marked as low credit risk even at the cost of losing some credit worthy candidates.

This is so because there is a potential of big money loss in the case when loan is approved to a low credit worthy candidate, however the profits from the income of such candidates is not much.

To priortize this business objective, we have to make sure that the ML models created are being able to recall the high credit risk class (class 1 in this case) even at the cost of suffering with poort recision.
In simple words - the errors where ML model identifies a high credit risk applicant as low risk applicant (False negatives) should be weighted more as compared to the False positives.

#### 1. Importing the required libraries

In [1]:
# Importing the packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')

#### 2. Loading and cleaning the dataset

In [2]:
# Loading the processed dataset
data = pd.read_csv('/content/combined_data.csv')

In [3]:
data.columns

Index(['Unnamed: 0', 'loan_application_id', 'applicant_id',
       'Months_loan_taken_for', 'Purpose', 'Principal_loan_amount',
       'EMI_rate_in_percentage_of_disposable_income', 'Property',
       'Has_coapplicant', 'Has_guarantor', 'Other_EMI_plans',
       'Number_of_existing_loans_at_this_bank', 'Loan_history',
       'high_risk_applicant', 'Primary_applicant_age_in_years', 'Gender',
       'Marital_status', 'Number_of_dependents', 'Housing',
       'Years_at_current_residence', 'Employment_status',
       'Has_been_employed_for_at_least', 'Has_been_employed_for_at_most',
       'Telephone', 'Foreign_worker', 'Savings_account_balance',
       'Balance_in_existing_bank_account_(lower_limit_of_bucket)',
       'Balance_in_existing_bank_account_(upper_limit_of_bucket)', 'other_emi',
       'employed_range', 'balance_range', 'Principal_loan_amount_log'],
      dtype='object')

In [4]:
# Used for feature selection

ignored_features = ['Other_EMI_plans', 'Has_been_employed_for_at_least', 'Has_been_employed_for_at_most', 'Telephone',
                   'balance_range','Principal_loan_amount','Number_of_dependents','loan_application_id', 'applicant_id',
                   'Balance_in_existing_bank_account_(lower_limit_of_bucket)', 'Balance_in_existing_bank_account_(upper_limit_of_bucket)']
cat_features = ['Purpose','Property', 'Loan_history', 'Gender', 'Marital_status', 
                'Housing', 'employed_range', 'Savings_account_balance', 'Employment_status', 'Years_at_current_residence',
               'Has_guarantor', 'Foreign_worker', 'Has_coapplicant'] # for dummy variables
ordinal_features = ['Number_of_existing_loans_at_this_bank', 'EMI_rate_in_percentage_of_disposable_income', 
                    'Years_at_current_residence']
numerical_features = ['Principal_loan_amount_log', 'Months_loan_taken_for', 'Primary_applicant_age_in_years']

In [5]:
print(data.shape)
print(len(ignored_features))
data.drop(ignored_features, axis=1, inplace=True)
print(data.shape)

(1000, 32)
11
(1000, 21)


In [6]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [7]:
label = 'high_risk_applicant'
feature_cols = data.columns.difference([label])

### Helper functions

Label encoding: assign each unique category in a categorical variable with an integer. No new columns are created. An example is shown below

![label_encoding.png](attachment:label_encoding.png)


#### 3. Encoding features using label encoder

For any categorical variable (dtype == object) with 2 unique categories, we will use label encoding, and for any categorical variable with more than 2 unique categories, we will use one-hot encoding.

For label encoding, we use the Scikit-Learn LabelEncoder and for one-hot encoding, the pandas get_dummies(df) function.

In [8]:
def get_encoded(feature_cols, data):
    """ 
    Returns data with encoded object features using label encoder
            Parameters:
                    feature_cols (list): List of column names present in data
                    data (dataframe): Dataframe containing daraset for training and testing
            Returns:
                    data (dataframe): Dataframe with object columns encoded using label encoder
    """
    le = LabelEncoder()
    le_count = 0

    # Iterate through the columns
    for col in feature_cols:
        if data[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(data[col].unique())) <= 2:
                print(col)
                # Train on the training data
                le.fit(data[col])
                # Transform both training and testing data
                data[col] = le.transform(data[col])

                # Keep track of how many columns were label encoded
                le_count += 1

    print('%d columns were label encoded.' % le_count)
    return data

#### 4. Printing mean classification scores for test dataset

In [44]:
def print_scores(scores_dict):
    """
    Prints ROC_AUC, mean precision, mean recall and mean f1 score for a cross validation score set
    """
    print('Mean ROC AUC: %.3f' % np.mean(scores_dict['test_roc_auc_sc']))
    print('Mean Precision: %.3f' % np.mean(scores_dict['test_precision_score']))
    print('Mean Recall: %.3f' % np.mean(scores_dict['test_reccall_score']))
    print('Mean f1 Score: %.3f' % np.mean(scores_dict['test_f1_score']))

#### 5. Dictionary containing all the scoring metrics

In [10]:
scoring = {'roc_auc_sc': 'roc_auc',
           'precision_score': 'precision',
           'reccall_score': 'recall',
           'f1_score': 'f1'}

### 6. Logistic Regression Model

**Model 1: Logistic Regression with all dataset and no class weight**

One-hot encoding: create a new column for each unique category in a categorical variable. Each observation recieves a 1 in the column for its corresponding category and a 0 in all other new columns.

![one_hot_encoding.png](attachment:one_hot_encoding.png)

In [11]:
data_encoded = get_encoded(feature_cols, data)
data_encoded = pd.get_dummies(data_encoded)

print(data_encoded.shape)
print(data_encoded.columns)

Gender
Housing
other_emi
3 columns were label encoded.
(1000, 43)
Index(['Months_loan_taken_for', 'EMI_rate_in_percentage_of_disposable_income',
       'Has_coapplicant', 'Has_guarantor',
       'Number_of_existing_loans_at_this_bank', 'high_risk_applicant',
       'Primary_applicant_age_in_years', 'Gender', 'Housing',
       'Years_at_current_residence', 'Foreign_worker', 'other_emi',
       'Principal_loan_amount_log', 'Purpose_FF&E', 'Purpose_None',
       'Purpose_business', 'Purpose_education', 'Purpose_electronic equipment',
       'Purpose_new vehicle', 'Purpose_used vehicle', 'Property_No Property',
       'Property_building society savings agreement/life insurance',
       'Property_car or other', 'Property_real estate',
       'Loan_history_critical/pending loans at other banks',
       'Loan_history_delay in paying off loans in the past',
       'Loan_history_existing loans paid back duly till now',
       'Loan_history_paid', 'Marital_status_divorced/separated/married',
   

In [12]:
label = 'high_risk_applicant'
feature_cols_model1 = data_encoded.columns.difference([label])

Splitting the data into training and testing set

In [13]:
train_X,test_X,train_y,test_y = train_test_split(data_encoded[feature_cols_model1], data_encoded[label], 
                                                 stratify=data_encoded[label], test_size = 0.2, random_state=42)

print('Size of training and testing set is:')
print(train_X.shape)
print(test_X.shape)

Size of training and testing set is:
(800, 42)
(200, 42)


Since many ML algorithms works well when the input features are standardised, i.e they vary from 0 to 1, we have used min max scaler below to transform the numerical features.

In [14]:
# Using min-max scaler to transform features from 0 to 1
train = train_X.copy()
test = test_X.copy()

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

**Using RepeatedStratifiedKFold:**

A single run of StratifiedKFold might result in a noisy estimate of model performance, as different splits of the data might result in very different results. That's where RepeatedStratifiedKFold comes into play.

RepeatedStratifiedKFold allows improving the estimated performance of a machine learning model, by simply repeating the cross-validation procedure multiple times (according to n_repeats value) and reporting the mean result across all folds from all runs.

**Using cross_validate from sklearn.model selection:**

cross_validate is used to valuate metrics by cross-validation. Important thing to note is that cross_val_score only allows to use one scoring metric. To calculate multiple such metrics, we are using cross_validate and passing a dictionary of all the scoring functions to be calculated.

In [15]:
log_reg = LogisticRegression(solver='lbfgs')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_validate(log_reg, train, train_y, scoring=scoring, cv=cv, n_jobs=-1)
print_scores(scores)

Mean ROC AUC: 0.741
Mean Precision: 0.574
Mean Recall: 0.358
Mean f1 Score: 0.437


**Analysis:**

- The model performs good as a baseline as it gives 0.742 AUC
- Though precision seems okay at 0.63, but the recall value is very less.
- Since the business objective is to minimize the cases of False Negatives, the goal directly translates to maximizing the recall.
- Recall is not very good.

**Model 2: Logistic Regression with all dataset and balanced class weight**

Since there are a lot more cases of low credit risk applicants as compared to high credit risk applicants, there is a class imbalance in the dataset as observed while doing EDA.

One way to deal with the class imbalance is to apply class_weight while initializing the model, so that mistakes made on data for higher weighted class is weighted more.

In [16]:
log_reg2 = LogisticRegression(solver='lbfgs', class_weight='balanced')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

scores = cross_validate(log_reg2, train, train_y, scoring=scoring, cv=cv, n_jobs=-1)
print_scores(scores)

Mean ROC AUC: 0.741
Mean Precision: 0.483
Mean Recall: 0.668
Mean f1 Score: 0.558


**Analysis:** 

- After applying the class weights, the recall increased from 0.36 to 0.64
- Overall AUC is 0.74 which is satisfactory.
- Precision suffered as the mistakes made on examples of class 0 are weighted less. 
- So overall, this classifer was not able to precisely label the class 1, but was able to label examples 1 when they were supposed to be 1 more frequently.
- This has led to decrease in False Negatives but has led to increase in False positives.

**Model 3: Logistic Regression with Feature selection and balanced class weight**

Important Features - Loan_history, other_emi, Property, Purpose, Housing, Savings_account_balance, Has_guarantor

Features to ignore - Gender, Marital_status, Employment_status, Number_of_existing_loans_at_this_bank, Number_of_dependents, Primary_applicant_age_in_years, Years_at_current_residence

In [17]:
imp_cols = ['Months_loan_taken_for', 'Purpose','EMI_rate_in_percentage_of_disposable_income', 'Property',
    'Has_guarantor','Loan_history','high_risk_applicant','Housing', 
    'Savings_account_balance','employed_range', 'Principal_loan_amount_log']

In [18]:
label = 'high_risk_applicant'
feature_cols_model3 = [w for w in feature_cols if w in imp_cols]
print(feature_cols_model3)

data_model3 = data[feature_cols_model3+[label]]

['EMI_rate_in_percentage_of_disposable_income', 'Has_guarantor', 'Housing', 'Loan_history', 'Months_loan_taken_for', 'Principal_loan_amount_log', 'Property', 'Purpose', 'Savings_account_balance', 'employed_range']


In [19]:
data_encoded = get_encoded(feature_cols_model3, data_model3)
data_dummy = pd.get_dummies(data_encoded)

0 columns were label encoded.


In [20]:
feature_cols_model3 = data_dummy.columns.difference([label])

print(data_dummy.shape)
print(data_dummy.columns)

(1000, 29)
Index(['EMI_rate_in_percentage_of_disposable_income', 'Has_guarantor',
       'Housing', 'Months_loan_taken_for', 'Principal_loan_amount_log',
       'high_risk_applicant',
       'Loan_history_critical/pending loans at other banks',
       'Loan_history_delay in paying off loans in the past',
       'Loan_history_existing loans paid back duly till now',
       'Loan_history_paid', 'Property_No Property',
       'Property_building society savings agreement/life insurance',
       'Property_car or other', 'Property_real estate', 'Purpose_FF&E',
       'Purpose_None', 'Purpose_business', 'Purpose_education',
       'Purpose_electronic equipment', 'Purpose_new vehicle',
       'Purpose_used vehicle', 'Savings_account_balance_High',
       'Savings_account_balance_Low', 'Savings_account_balance_None',
       'employed_range_0 year - 1 year', 'employed_range_1 year - 4 years',
       'employed_range_4 years - 7 years', 'employed_range_7 years and up',
       'employed_range_None'

In [21]:
train_X,test_X,train_y,test_y = train_test_split(data_dummy[feature_cols_model3], data_dummy[label], 
                                                 stratify=data_dummy[label], test_size = 0.2, random_state=42)
print(train_X.shape)
print(test_X.shape)

train = train_X.copy()
test = test_X.copy()

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

(800, 28)
(200, 28)


In [22]:
log_reg3 = LogisticRegression(solver='lbfgs', class_weight='balanced')
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_validate(log_reg3, train, train_y, scoring=scoring, cv=cv, n_jobs=-1)

print_scores(scores)

Mean ROC AUC: 0.742
Mean Precision: 0.469
Mean Recall: 0.642
Mean f1 Score: 0.539


**Analysis:**

- A little improvement in Area under the ROC curve.
- Precision and recall value has almost been the same
- f1 score has declined.
- This version is not an improvement of model2

### 7. Evaluating Logistic Regression Models

The best model out of the above 3 models trained Model 2 trained on the entire dataset with class weights being balanced.

Intuitively, this is helping because assigning higher weights to the makes caused while doing wrong predictions for label 1. 

In [23]:
data_encoded = get_encoded(feature_cols, data)
data_encoded = pd.get_dummies(data_encoded)

print(feature_cols)

0 columns were label encoded.
Index(['EMI_rate_in_percentage_of_disposable_income', 'Employment_status',
       'Foreign_worker', 'Gender', 'Has_coapplicant', 'Has_guarantor',
       'Housing', 'Loan_history', 'Marital_status', 'Months_loan_taken_for',
       'Number_of_existing_loans_at_this_bank',
       'Primary_applicant_age_in_years', 'Principal_loan_amount_log',
       'Property', 'Purpose', 'Savings_account_balance',
       'Years_at_current_residence', 'employed_range', 'other_emi'],
      dtype='object')


In [24]:
label = 'high_risk_applicant'
feature_cols_model1 = data_encoded.columns.difference([label])

train_X,test_X,train_y,test_y = train_test_split(data_encoded[feature_cols_model1], data_encoded[label], 
                                                 stratify=data_encoded[label], test_size = 0.2, random_state=42)

print('Size of training and testing set is:')
print(train_X.shape)
print(test_X.shape)

train = train_X.copy()
test = test_X.copy()

scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

Size of training and testing set is:
(800, 42)
(200, 42)


In [25]:
log_reg2 = LogisticRegression(solver='lbfgs', class_weight='balanced')
log_reg2.fit(train, train_y)
pred = log_reg2.predict(test)

In [26]:
print(confusion_matrix(test_y, pred))
print(classification_report(test_y, pred))

[[96 44]
 [20 40]]
              precision    recall  f1-score   support

           0       0.83      0.69      0.75       140
           1       0.48      0.67      0.56        60

    accuracy                           0.68       200
   macro avg       0.65      0.68      0.65       200
weighted avg       0.72      0.68      0.69       200



### 8. Random Forest

**Model 4: Random Forest with all dataset and no class weight**

In [27]:
data_encoded = get_encoded(feature_cols, data)
data_encoded = pd.get_dummies(data_encoded)

print(data_encoded.shape)
print(data_encoded.columns)

0 columns were label encoded.
(1000, 43)
Index(['Months_loan_taken_for', 'EMI_rate_in_percentage_of_disposable_income',
       'Has_coapplicant', 'Has_guarantor',
       'Number_of_existing_loans_at_this_bank', 'high_risk_applicant',
       'Primary_applicant_age_in_years', 'Gender', 'Housing',
       'Years_at_current_residence', 'Foreign_worker', 'other_emi',
       'Principal_loan_amount_log', 'Purpose_FF&E', 'Purpose_None',
       'Purpose_business', 'Purpose_education', 'Purpose_electronic equipment',
       'Purpose_new vehicle', 'Purpose_used vehicle', 'Property_No Property',
       'Property_building society savings agreement/life insurance',
       'Property_car or other', 'Property_real estate',
       'Loan_history_critical/pending loans at other banks',
       'Loan_history_delay in paying off loans in the past',
       'Loan_history_existing loans paid back duly till now',
       'Loan_history_paid', 'Marital_status_divorced/separated/married',
       'Marital_status_marri

In [28]:
label = 'high_risk_applicant'
feature_cols_model1 = data_encoded.columns.difference([label])

Splitting the data into training and testing set

In [29]:
train_X,test_X,train_y,test_y = train_test_split(data_encoded[feature_cols_model1], data_encoded[label], 
                                                 stratify=data_encoded[label], test_size = 0.2, random_state=42)

print('Size of training and testing set is:')
print(train_X.shape)
print(test_X.shape)

train = train_X.copy()
test = test_X.copy()

Size of training and testing set is:
(800, 42)
(200, 42)


Performing grid search for best parameters

In [30]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [31]:
# Use the random grid to search for best hyperparameters

#Initialize the random forest model
random_forest = RandomForestClassifier(random_state = 42, verbose = 0, n_jobs = -1)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = random_forest, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=1)

In [32]:
print(rf_random.best_params_)

best_random = rf_random.best_estimator_
preds = best_random.predict(test)

print(confusion_matrix(test_y, preds))
print(classification_report(test_y, preds))

{'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 60, 'bootstrap': False}
[[129  11]
 [ 33  27]]
              precision    recall  f1-score   support

           0       0.80      0.92      0.85       140
           1       0.71      0.45      0.55        60

    accuracy                           0.78       200
   macro avg       0.75      0.69      0.70       200
weighted avg       0.77      0.78      0.76       200



**Analysis:**

- The model performs well for unbalanced class weights.
- It has a decent recall of 0.45 for class 1 and a good precision for both the clases.

**Model 5: Random Forest with all dataset and balanced class weight**

In [33]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [34]:
# Use the random grid to search for best hyperparameters

#Initialize the random forest model
random_forest2 = RandomForestClassifier(random_state = 42, verbose = 0, n_jobs = -1,
                                       class_weight='balanced')

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random2 = RandomizedSearchCV(estimator = random_forest2, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random2.fit(train, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    n_jobs=-1,
                                                    random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2

In [35]:
print(rf_random2.best_params_)

best_random2 = rf_random2.best_estimator_
preds2 = best_random2.predict(test)

print(confusion_matrix(test_y, preds2))
print(classification_report(test_y, preds2))

{'n_estimators': 600, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'max_depth': 70, 'bootstrap': False}
[[113  27]
 [ 30  30]]
              precision    recall  f1-score   support

           0       0.79      0.81      0.80       140
           1       0.53      0.50      0.51        60

    accuracy                           0.71       200
   macro avg       0.66      0.65      0.66       200
weighted avg       0.71      0.71      0.71       200



**Analysis:** 

- Some improvement in the recall score for class 1 due to class weights balancing
- Precision of label 1 got reduced.

**Model 6: Random Forest with Feature selection and balanced class weight**

Important Features - Loan_history, other_emi, Property, Purpose, Housing, Savings_account_balance, Has_guarantor

Features to ignore - Gender, Marital_status, Employment_status, Number_of_existing_loans_at_this_bank, Number_of_dependents, Primary_applicant_age_in_years, Years_at_current_residence

In [36]:
imp_cols = ['Months_loan_taken_for', 'Purpose','EMI_rate_in_percentage_of_disposable_income', 'Property',
    'Has_guarantor','Loan_history','high_risk_applicant','Housing', 
    'Savings_account_balance','employed_range', 'Principal_loan_amount_log']

In [37]:
label = 'high_risk_applicant'
feature_cols_model6 = [w for w in feature_cols if w in imp_cols]
print(feature_cols_model6)

data_model6 = data[feature_cols_model6+[label]]

['EMI_rate_in_percentage_of_disposable_income', 'Has_guarantor', 'Housing', 'Loan_history', 'Months_loan_taken_for', 'Principal_loan_amount_log', 'Property', 'Purpose', 'Savings_account_balance', 'employed_range']


In [38]:
data_encoded = get_encoded(feature_cols_model6, data_model6)
data_dummy = pd.get_dummies(data_encoded)

0 columns were label encoded.


In [39]:
feature_cols_model6 = data_dummy.columns.difference([label])

print(data_dummy.shape)
print(data_dummy.columns)

(1000, 29)
Index(['EMI_rate_in_percentage_of_disposable_income', 'Has_guarantor',
       'Housing', 'Months_loan_taken_for', 'Principal_loan_amount_log',
       'high_risk_applicant',
       'Loan_history_critical/pending loans at other banks',
       'Loan_history_delay in paying off loans in the past',
       'Loan_history_existing loans paid back duly till now',
       'Loan_history_paid', 'Property_No Property',
       'Property_building society savings agreement/life insurance',
       'Property_car or other', 'Property_real estate', 'Purpose_FF&E',
       'Purpose_None', 'Purpose_business', 'Purpose_education',
       'Purpose_electronic equipment', 'Purpose_new vehicle',
       'Purpose_used vehicle', 'Savings_account_balance_High',
       'Savings_account_balance_Low', 'Savings_account_balance_None',
       'employed_range_0 year - 1 year', 'employed_range_1 year - 4 years',
       'employed_range_4 years - 7 years', 'employed_range_7 years and up',
       'employed_range_None'

In [45]:
train_X,test_X,train_y,test_y = train_test_split(data_dummy[feature_cols_model6], data_dummy[label], 
                                                 stratify=data_dummy[label], test_size = 0.2, random_state=42)
print(train_X.shape)
print(test_X.shape)

train = train_X.copy()
test = test_X.copy()

(800, 28)
(200, 28)


In [41]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [42]:
# Use the random grid to search for best hyperparameters

#Initialize the random forest model
random_forest3 = RandomForestClassifier(random_state = 42, verbose = 0, n_jobs = -1,
                                       class_weight='balanced')

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random3 = RandomizedSearchCV(estimator = random_forest3, param_distributions = random_grid, n_iter = 100, 
                               cv = 3, verbose=1, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random3.fit(train, train_y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(class_weight='balanced',
                                                    n_jobs=-1,
                                                    random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2

In [43]:
print(rf_random3.best_params_)

best_random3 = rf_random3.best_estimator_
preds3 = best_random3.predict(test)

print(confusion_matrix(test_y, preds3))
print(classification_report(test_y, preds3))

{'n_estimators': 800, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}
[[107  33]
 [ 27  33]]
              precision    recall  f1-score   support

           0       0.80      0.76      0.78       140
           1       0.50      0.55      0.52        60

    accuracy                           0.70       200
   macro avg       0.65      0.66      0.65       200
weighted avg       0.71      0.70      0.70       200



**Analysis:**

- Further improvement in the recall of class 1
- Precision has suffered.
- Overall this is the best random forest model out of the three random forest models trained.

In [49]:
data

,Months_loan_taken_for,Purpose,EMI_rate_in_percentage_of_disposable_income,Property,Has_coapplicant,Has_guarantor,Number_of_existing_loans_at_this_bank,Loan_history,high_risk_applicant,Primary_applicant_age_in_years,Gender,Marital_status,Housing,Years_at_current_residence,Employment_status,Foreign_worker,Savings_account_balance,other_emi,employed_range,Principal_loan_amount_log
0,6,electronic equipment,4,real estate,0,0,2,critical/pending loans at other banks,0,67,1,single,1,4,skilled employee / official,1,None,0,7 years and up,13.971659
1,48,electronic equipment,2,real estate,0,0,1,existing loans paid back duly till now,1,22,0,divorced/separated/married,1,2,skilled employee / official,1,Low,0,1 year - 4 years,15.599070
2,12,education,2,real estate,0,0,1,critical/pending loans at other banks,0,49,1,single,1,3,unskilled - resident,1,Low,0,4 years - 7 years,14.555541
3,42,FF&E,2,building society savings agreement/life insurance,0,1,1,existing loans paid back duly till now,0,45,1,single,0,4,skilled employee / official,1,Low,0,4 years - 7 years,15.880092
4,24,new vehicle,3,No Property,0,0,2,delay in paying off loans in the past,1,53,1,single,0,4,skilled employee / official,1,Low,0,1 year - 4 years,15.398604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,FF&E,3,real estate,0,0,1,existing loans paid back duly till now,0,31,0,divorced/separated/married,1,4,unskilled - resident,1,Low,0,4 years - 7 years,14.367094
996,30,used vehicle,4,building society savings agreement/life insurance,0,0,1,existing loans paid back duly till now,0,40,1,divorced/separated/married,1,4,management / self-employed / highly qualified ...,1,Low,0,1 year - 4 years,15.165400
997,12,electronic equipment,4,car or other,0,0,1,existing loans paid back duly till now,0,38,1,single,1,4,skilled employee / official,1,Low,0,7 years and up,13.597355
998,45,electronic equipment,4,No Property,0,0,1,existing loans paid back duly till now,1,23,1,single,0,4,skilled employee / official,1,Low,0,1 year - 4 years,14.427990


In [50]:
print(train.shape)
print(test.shape)

data.to_csv('train.csv', index=False)
data.to_csv('test.csv', index=False)

(800, 28)
(200, 28)


In [ ]:
#!pip install h2o

In [56]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()
# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("/content/train.csv")
test = h2o.import_file("/content/test.csv")
# Identify the response and set of predictors
y = "high_risk_applicant"
x = list(train.columns)  #if x is defined as all columns except the response, then x is not required
x.remove(y)
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
# Run AutoML for 30 seconds
aml = H2OAutoML(max_runtime_secs = 30)
aml.train(x = x, y = y, training_frame = train)
# Print Leaderboard (ranked by xval metrics)
aml.leaderboard
# (Optional) Evaluate performance on a test set
perf = aml.leader.model_performance(test)
perf.auc()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 mins 15 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,21 days and 50 minutes
H2O_cluster_name:,H2O_from_python_unknownUser_vskmiq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


0.918854761904762

In [57]:
aml = H2OAutoML(max_runtime_secs=30)
# Launch an AutoML run
aml.train(y=y, training_frame=train)
# Get the best model in the AutoML Leaderboard
aml.leader

# Get AutoML object by `project_name`
get_aml = h2o.automl.get_automl(aml.project_name)
# Get the best model in the AutoML Leaderboard
get_aml.leader

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_BestOfFamily_3_AutoML_2_20221117_164838

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.12295807965329422
RMSE: 0.35065378887628496
LogLoss: 0.3981086016151928
AUC: 0.9226952380952381
AUCPR: 0.8507774884044272
Gini: 0.8453904761904762
Null degrees of freedom: 999
Residual degrees of freedom: 994
Null deviance: 1221.7286041097877
Residual deviance: 796.2172032303856
AIC: 808.2172032303856

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.38795458018411666
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      619  81   0.1157   (81.0/700.0)
1      67   233  0.2233   (67.0/300.0)
Total  686  314  0.148    (148.0/1000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.387955     0.758958  168
max f2                       0.268509     0.834788  240
max f0point5                 0.4545       0.791396  133
max accuracy                 0.4545       0.857     133
max precision                0.949666     1         0
max recall                   0.145111     1         324
max specificity              0.949666     1         0
max absolute_mcc             0.395857     0.653049  163
max min_per_class_accuracy   0.357591     0.834286  186
max mean_per_class_accuracy  0.331383     0.842619  201
max tns                      0.949666     700       0
max fns                      0.949666     299       0
max fps                      0.0144714    700       399
max tps                      0.145111     300       324
max tnr                      0.949666     1         0
max fnr                      0.949666     0.996667  0
max fpr                      0.0144714    1         399
max tpr                      0.145111     1         324

Gains/Lift Table: Avg response rate: 30.00 %, avg score: 31.28 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        0.794287           3.33333    3.33333            1                0.859065   1                           0.859065            0.0333333       0.0333333                  233.333   233.333            0.0333333
2        0.02                        0.754396           3.33333    3.33333            1                0.776446   1                           0.817756            0.0333333       0.0666667                  233.333   233.333            0.0666667
3        0.03                        0.735181           3.33333    3.33333            1                0.744873   1                           0.793462            0.0333333       0.1                        233.333   233.333            0.1
4        0.04                        0.716314           3.33333    3.33333            1                0.727053   1                           0.77686             0.0333333       0.133333                   233.333   233.333            0.133333
5        0.05                        0.700607           3.33333    3.33333            1                0.708166   1                           0.763121            0.0333333       0.166667                   233.333   233.333            0.166667
6        0.1                         0.603624           3.06667    3.2                0.92             0.651659   0.96                        0.70739             0.153333        0.32                       206.667   220    

In [60]:
# Get leaderboard with all possible columns
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_BestOfFamily_3_AutoML_2_20221117_164838,0.751271,0.525061,0.572752,0.30619,0.417638,0.174421,492,0.137954,StackedEnsemble
StackedEnsemble_AllModels_1_AutoML_2_20221117_164838,0.750493,0.524853,0.567518,0.311667,0.418034,0.174753,521,0.105198,StackedEnsemble
StackedEnsemble_BestOfFamily_2_AutoML_2_20221117_164838,0.749048,0.527982,0.569672,0.31,0.41805,0.174766,473,0.082025,StackedEnsemble
GLM_1_AutoML_2_20221117_164838,0.746471,0.527971,0.556883,0.306905,0.419288,0.175802,183,0.066254,GLM
StackedEnsemble_AllModels_2_AutoML_2_20221117_164838,0.745781,0.528284,0.561117,0.315476,0.420044,0.176437,480,0.103242,StackedEnsemble
StackedEnsemble_BestOfFamily_1_AutoML_2_20221117_164838,0.741717,0.531195,0.5545,0.31,0.421079,0.177308,351,0.050447,StackedEnsemble
GBM_2_AutoML_2_20221117_164838,0.735052,0.534021,0.56554,0.317619,0.421701,0.177832,340,0.039365,GBM
XGBoost_3_AutoML_2_20221117_164838,0.731705,0.537234,0.549592,0.332143,0.423997,0.179774,367,0.010075,XGBoost
GBM_1_AutoML_2_20221117_164838,0.729424,0.542002,0.519565,0.311905,0.42706,0.18238,364,0.032736,GBM
GBM_3_AutoML_2_20221117_164838,0.725974,0.544783,0.528043,0.33119,0.427443,0.182707,308,0.03255,GBM


In [63]:
# Get the best model using the metric
m = aml.leader
# this is equivalent to
m = aml.get_best_model()

# Get the best model using a non-default metric
m = aml.get_best_model(criterion="logloss")
m

Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_AllModels_1_AutoML_2_20221117_164838

No summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.11283290308616041
RMSE: 0.33590609266007726
LogLoss: 0.36908943723606036
AUC: 0.9340285714285715
AUCPR: 0.8718620455606518
Gini: 0.8680571428571431
Null degrees of freedom: 999
Residual degrees of freedom: 995
Null deviance: 1221.7286041097877
Residual deviance: 738.1788744721207
AIC: 748.1788744721207

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3721281438436489
       0    1    Error    Rate
-----  ---  ---  -------  --------------
0      624  76   0.1086   (76.0/700.0)
1      58   242  0.1933   (58.0/300.0)
Total  682  318  0.134    (134.0/1000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.372128     0.783172  174
max f2                       0.248897     0.846108  243
max f0point5                 0.486499     0.811404  124
max accuracy                 0.418309     0.871     154
max precision                0.963197     1         0
max recall                   0.129526     1         320
max specificity              0.963197     1         0
max absolute_mcc             0.380034     0.687527  171
max min_per_class_accuracy   0.343879     0.843333  190
max mean_per_class_accuracy  0.313007     0.853333  207
max tns                      0.963197     700       0
max fns                      0.963197     298       0
max fps                      0.00898604   700       399
max tps                      0.129526     300       320
max tnr                      0.963197     1         0
max fnr                      0.963197     0.993333  0
max fpr                      0.00898604   1         399
max tpr                      0.129526     1         320

Gains/Lift Table: Avg response rate: 30.00 %, avg score: 29.92 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        0.835326           3.33333    3.33333            1                0.883735   1                           0.883735            0.0333333       0.0333333                  233.333   233.333            0.0333333
2        0.02                        0.792918           3.33333    3.33333            1                0.810033   1                           0.846884            0.0333333       0.0666667                  233.333   233.333            0.0666667
3        0.03                        0.771651           3.33333    3.33333            1                0.783694   1                           0.825821            0.0333333       0.1                        233.333   233.333            0.1
4        0.04                        0.751142           3.33333    3.33333            1                0.760632   1                           0.809524            0.0333333       0.133333                   233.333   233.333            0.133333
5        0.05                        0.72973            3.33333    3.33333            1                0.741174   1                           0.795854            0.0333333       0.166667                   233.333   233.333            0.166667
6        0.1                         0.624002           3.2        3.26667            0.96             0.677838   0.98                        0.736846            0.16            0.326667                   220       226.667   

Hence we can see in the leaderboard that the best model is StackedEnsemble having all perfect scores on test data too